In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from src.network_diffusion_model import NDM
from src.FKPP_model import FKPP
from src.find_optimal_timepoint import find_optimal_timepoint
from time import time

In [3]:
pd.read_csv("../simulated_data/RegionList.csv")

,Label
0,Bankssts_L
1,Caudalanteriorcingulate_L
2,Caudalmiddlefrontal_L
3,Cuneus_L
4,Entorhinal_L
...,...
79,Frontalpole_R
80,Temporalpole_R
81,Transversetemporal_R
82,Insula_R


In [ ]:
NDM(conn)